# RAF-DB Dataset

In [2]:
# trying to fix cv2 numpy 2> incompatable problem
%pip install "numpy<2.0" 

  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import kagglehub
import matplotlib.pyplot as plt
import kagglehub

In [4]:
# Download two datasets
path_raf = kagglehub.dataset_download("shuvoalok/raf-db-dataset")
print("RAF‑DB dataset path:", path_raf)

path_fer = kagglehub.dataset_download("msambare/fer2013")
print("FER‑2013 dataset path:", path_fer)

RAF‑DB dataset path: /Users/samlin/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2
FER‑2013 dataset path: /Users/samlin/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [ ]:
# not sure why we need dataset path exactly, if alr stored in path_raf
# also I get a FileNotFound with dataset_path_raf
# dataset_path_raf = '/root/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2/'

# all the 'path_raf's below used to be dataset_path_raf

train_labels_csv_raf = os.path.join(path_raf, 'train_labels.csv')
test_labels_csv_raf = os.path.join(path_raf, 'test_labels.csv')

images_set_raf = os.path.join(path_raf, 'DATASET')

images_train_set_raf = os.path.join(images_set_raf, 'train')
images_test_set_raf = os.path.join(images_set_raf, 'test')

In [12]:
# RAF labels
# 1 = Surprise
# 2 = Fear
# 3 = Disgust
# 4 = Happy
# 5 = Sad
# 6 = Angry
# 7 = Neutral

# Choose 100x100 for both target datasets
target_size = (100, 100)

# Translate RAF labels to match FER's style
raf_to_fer = {
  '1': '5',   # Surprise
  '2': '2',   # Fear
  '3': '1',   # Disgust
  '4': '3',   # Happy
  '5': '4',   # Sad
  '6': '0',   # Angry
  '7': '6'    # Neutral
}

In [ ]:
# Load RAF training labels CSV
train_labels_df_raf = pd.read_csv(train_labels_csv_raf)

# Load RAF training images
raf_train_images = []
raf_train_labels = []

for idx, row in train_labels_df_raf.iterrows():
  img_filename = row['image']
  original_label = str(row['label'])

  # Translate label
  if original_label not in raf_to_fer:
    print(f"Cannot find RAF label: {original_label}")
    continue
  mapped_label = raf_to_fer[original_label]

  img_path = os.path.join(images_train_set_raf, original_label, img_filename)
  img = cv2.imread(img_path)

  # Resize and normalize image
  img = cv2.resize(img, target_size)
  img = img.astype('float32') / 255.0

  raf_train_images.append(img)
  raf_train_labels.append(mapped_label)

X_raf_train = np.array(raf_train_images)
y_raf_train = np.array(raf_train_labels)
print("RAF‑DB training images shape:", X_raf_train.shape)

RAF‑DB training images shape: (12271, 100, 100, 3)


In [14]:
raf_test_images = []
raf_test_labels = []

test_labels_df_raf = pd.read_csv(test_labels_csv_raf)
for idx, row in test_labels_df_raf.iterrows():
  img_filename = row['image']
  original_label = str(row['label'])
  if original_label not in raf_to_fer:
    print(f"CAnnot find RAF label: {original_label}")
    continue
  mapped_label = raf_to_fer[original_label]

  img_path = os.path.join(images_test_set_raf, original_label, img_filename)
  img = cv2.imread(img_path)

  img = cv2.resize(img, target_size)
  img = img.astype('float32') / 255.0

  raf_test_images.append(img)
  raf_test_labels.append(mapped_label)

X_raf_test = np.array(raf_test_images)
y_raf_test = np.array(raf_test_labels)
print("RAF‑DB test images shape:", X_raf_test.shape)

# Combine training and test sets of RAF
X_raf = np.concatenate((X_raf_train, X_raf_test), axis=0)
y_raf = np.concatenate((y_raf_train, y_raf_test), axis=0)
print("Combined RAF‑DB images shape:", X_raf.shape)

RAF‑DB test images shape: (3068, 100, 100, 3)
Combined RAF‑DB images shape: (15339, 100, 100, 3)


# FER-2013 Dataset

In [18]:
# same thing about the dataset_path_fer with path_fer
# dataset_path_fer = '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1'

images_train_set_fer = os.path.join(path_fer, 'train')
images_test_set_fer = os.path.join(path_fer, 'test')

In [19]:
# FER Labels
# 0 = Angry
# 1 = Disgust
# 2 = Fear
# 3 = Happy
# 4 = Sad
# 5 = Surprise
# 6 = Neutral

CATEGORIES_FER = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Map folder names to unified labels:
fer_mapping = {
  'angry': '0',
  'disgust': '1',
  'fear': '2',
  'happy': '3',
  'sad': '4',
  'surprise': '5',
  'neutral': '6'
}

def resize_image(img, target_size):
  return cv2.resize(img, target_size)

In [20]:
fer_train_images = []
fer_train_labels = []

for category in CATEGORIES_FER:
  folder = os.path.join(images_train_set_fer, category)

  for img_file in os.listdir(folder):
    img_path = os.path.join(folder, img_file)
    img = cv2.imread(img_path)
    if img is None:
      print(f"Failed to load image: {img_path}")
      continue

    # FER images are originally 48x48; resize to 100x100 to be consistent with RAF
    img = resize_image(img, target_size)
    img = img.astype('float32') / 255.0
    fer_train_images.append(img)
    fer_train_labels.append(category)

X_fer_train = np.array(fer_train_images)
y_fer_train = np.array(fer_train_labels)
print("FER‑2013 training images shape:", X_fer_train.shape)

FER‑2013 training images shape: (28709, 100, 100, 3)


In [21]:
fer_test_images = []
fer_test_labels = []

for category in CATEGORIES_FER:
  folder = os.path.join(images_test_set_fer, category)

  for img_file in os.listdir(folder):
    img_path = os.path.join(folder, img_file)
    img = cv2.imread(img_path)
    if img is None:
      print(f"Failed to load image: {img_path}")
      continue

    img = resize_image(img, target_size)
    img = img.astype('float32') / 255.0
    fer_test_images.append(img)
    fer_test_labels.append(category)

X_fer_test = np.array(fer_test_images)
y_fer_test = np.array(fer_test_labels)
print("FER‑2013 test images shape:", X_fer_test.shape)

# Combine FER training and test sets
X_fer = np.concatenate((X_fer_train, X_fer_test), axis=0)
y_fer = np.concatenate((y_fer_train, y_fer_test), axis=0)
print("Combined FER‑2013 images shape:", X_fer.shape)

FER‑2013 test images shape: (7178, 100, 100, 3)
Combined FER‑2013 images shape: (35887, 100, 100, 3)


In [22]:
X_combined_total = np.concatenate((X_raf, X_fer), axis=0)
y_combined_total = np.concatenate((y_raf, y_fer), axis=0)
print("\nTotal combined images shape:", X_combined_total.shape)
print("Total combined labels shape:", y_combined_total.shape)


Total combined images shape: (51226, 100, 100, 3)
Total combined labels shape: (51226,)


In [23]:
X_final_train, X_final_test, y_final_train, y_final_test = train_test_split(
  X_combined_total, y_combined_total, test_size=0.2, random_state=42, shuffle=True, stratify=y_combined_total
)

print("\nFinal Training set shape:", X_final_train.shape, y_final_train.shape)
print("Final Test set shape:", X_final_test.shape, y_final_test.shape)


Final Training set shape: (40980, 100, 100, 3) (40980,)
Final Test set shape: (10246, 100, 100, 3) (10246,)


In [ ]:
# was able to get same exact results for all output above

-------

### Implementation

In [51]:
import torch
from torch import nn, optim
import torch.nn.functional as F

In [69]:
y_final_train[0]

np.str_('happy')

In [79]:
# need to make all labels into ints, aligning with FER-2013 labeling
fer_mapping = { # already defined above but redefined here for convenience
  'angry': '0',
  'disgust': '1',
  'fear': '2',
  'happy': '3',
  'sad': '4',
  'surprise': '5',
  'neutral': '6'
}

def to_fer_labels(labels):
    for i in range(len(labels)):
        if labels[i] in fer_mapping.keys():
            labels[i] = fer_mapping[labels[i]]
        
to_fer_labels(y_final_train)

In [80]:
y_final_train[:5]

array(['3', '6', '0', '3', '2'], dtype='<U8')

In [83]:
# before implementation: need to split dataset again for happy-sad and all the other emotions
# RAF-DB is currently using FER-2013 label system

def split_datasets(images, labels):
    pcnn_images, pcnn_labels = [], []
    scnn_images, scnn_labels = [], []

    for img, lbl in zip(images, labels):
        if lbl == '3':   # Happy
            pcnn_images.append(img)
            pcnn_labels.append(0)
        elif lbl == '4':  # Sad
            pcnn_images.append(img)
            pcnn_labels.append(1)
        elif lbl in ['5', '6', '0', '2']:  # Surprise, Neutral, Angry, Fear
            mapped_label = {'5':'0', '6':'1', '0':'2', '2':'3'}[lbl]
            scnn_images.append(img)
            scnn_labels.append(mapped_label)

    return (pcnn_images, pcnn_labels), (scnn_images, scnn_labels)

# split RAF-DB
(pcnn_train_imgs, pcnn_train_labels), _ = split_datasets(X_final_train, y_final_train)
(pcnn_test_imgs, pcnn_test_labels), _ = split_datasets(X_final_test, y_final_test)

In [ ]:
## implementation of networks

In the proposed CNN model, each convolution layer contains a rectifed linear
activation unit (relu) activation function to apply stochastic gradient descent with
backpropagation of errors for model training. 

After the convolution and max-pooling layers, the network includes a fully connected dense layer.

To overcome overftting when training, the regularisation technique
of dropout is applied after every dense layer with a rate of 0.2 

At the end of the final dense layer used for classifcation, a softmax activation function is applied that
converts the component values into probabilities that sum to 1 by generating a vector
that contains the probability distribution of the potential outcomes. 

For optimizing the training network and improving the results, the adaptive moment estimation (Adam)
optimiser is applied during model compilation

### P-CNN 
First, a primary CNN (P-CNN) model architecture is designed to classify the image
based on the two primary emotions of showing happy or sad. 

This P-CNN contains three convolution layers and three fully connected layers with 1024 neurons each.
Among the three fully connected layers, the final layer is a two-neuron layer with the
softmax function to apply the classification.

In [85]:
# P-CNN

class PCNN(nn.Module):
    def __init__(self):
        super(PCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Assuming RGB input
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 12 * 12, 1024)  # Adjust input size according to image dims
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 2)  # happy or sad

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x


### S-CNN
The secondary CNN (S-CNN) model is designed to perform a classification
based on the sub-category of emotions from the results of the P-CNN. 

The S-CNN
is designed with five 2D convolution layers, each attached to the max-pooling layer.


After the convolution layers, three fully connected layers with 1024 neurons each are
included. 

To avoid overfitting and reduce the training time, two dropout layers with the
rate of 0.2 are introduced into the network after the first and second dense layers

In [113]:
class SCNN(nn.Module):
    def __init__(self):
        super(SCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(512 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 4)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 100 -> 50
        x = self.pool(F.relu(self.conv2(x)))  # 50 -> 25
        x = self.pool(F.relu(self.conv3(x)))  # 25 -> 12
        x = self.pool(F.relu(self.conv4(x)))  # 12 -> 6
        x = self.pool(F.relu(self.conv5(x)))  # 6 -> 3
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        return self.fc3(x)


In [87]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        x = torch.tensor(self.images[idx]).permute(2, 0, 1).float()  # HWC → CHW
        y = torch.tensor(self.labels[idx]).long()
        return x, y

In [88]:
def train_model(model, dataloader, criterion, optimizer, device, epochs=10):
    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        correct = 0

        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            correct += (out.argmax(1) == y).sum().item()

        acc = correct / len(dataloader.dataset)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader.dataset):.4f}, Accuracy: {acc:.4f}")

In [104]:
from torch.utils.data import DataLoader

pcnn_train_ds = EmotionDataset(pcnn_train_imgs, pcnn_train_labels)
pcnn_test_ds = EmotionDataset(pcnn_test_imgs, pcnn_test_labels)

pcnn_train_loader = DataLoader(pcnn_train_ds, batch_size=64, shuffle=True)
pcnn_test_loader = DataLoader(pcnn_test_ds, batch_size=64, shuffle=False)

model = PCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_model(model, pcnn_train_loader, criterion, optimizer, device, epochs=20)


Epoch 1, Loss: 0.5713, Accuracy: 0.6962
Epoch 2, Loss: 0.4400, Accuracy: 0.7855
Epoch 3, Loss: 0.3786, Accuracy: 0.8212
Epoch 4, Loss: 0.3310, Accuracy: 0.8495
Epoch 5, Loss: 0.2867, Accuracy: 0.8707
Epoch 6, Loss: 0.2376, Accuracy: 0.9005
Epoch 7, Loss: 0.1846, Accuracy: 0.9265
Epoch 8, Loss: 0.1413, Accuracy: 0.9462
Epoch 9, Loss: 0.1054, Accuracy: 0.9599
Epoch 10, Loss: 0.0806, Accuracy: 0.9686
Epoch 11, Loss: 0.0641, Accuracy: 0.9762
Epoch 12, Loss: 0.0576, Accuracy: 0.9789
Epoch 13, Loss: 0.0476, Accuracy: 0.9828
Epoch 14, Loss: 0.0425, Accuracy: 0.9856
Epoch 15, Loss: 0.0353, Accuracy: 0.9881
Epoch 16, Loss: 0.0361, Accuracy: 0.9873
Epoch 17, Loss: 0.0321, Accuracy: 0.9891
Epoch 18, Loss: 0.0257, Accuracy: 0.9911
Epoch 19, Loss: 0.0318, Accuracy: 0.9892
Epoch 20, Loss: 0.0241, Accuracy: 0.9912


In [105]:
# def evaluate(model, dataloader, device):
#     model.eval()
#     correct = 0
#     with torch.no_grad():
#         for x, y in dataloader:
#             x, y = x.to(device), y.to(device)
#             preds = model(x).argmax(1)
#             correct += (preds == y).sum().item()
#     acc = correct / len(dataloader.dataset)
#     print(f"Validation Accuracy: {acc:.4f}")
#     model.train()
    
# # put this in train_model loop for evaluation
# evaluate(model, pcnn_test_loader, device)

In [106]:
# ReLU and SGD
# conv and max pool, then one? FC
# after FC is softmax
# ?? what's model compilation: Adam as optimizer


In [108]:
pcnn_model = model

In [109]:
model

PCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=18432, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [114]:
_, (scnn_train_imgs, scnn_train_labels) = split_datasets(X_final_train, y_final_train)
_, (scnn_test_imgs, scnn_test_labels) = split_datasets(X_final_test, y_final_test)

scnn_train_labels = list(map(int, scnn_train_labels))
scnn_test_labels = list(map(int, scnn_test_labels))


scnn_train_ds = EmotionDataset(scnn_train_imgs, scnn_train_labels)
scnn_test_ds = EmotionDataset(scnn_test_imgs, scnn_test_labels)

scnn_train_loader = DataLoader(scnn_train_ds, batch_size=64, shuffle=True)
scnn_test_loader = DataLoader(scnn_test_ds, batch_size=64, shuffle=False)


In [115]:
scnn_train_labels[0]

1

In [140]:
scnn_test_labels[:10]

[2, 1, 0, 2, 1, 0, 1, 0, 1, 0]

In [121]:
scnn_model = SCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(scnn_model.parameters(), lr=1e-3)

train_model(scnn_model, scnn_train_loader, criterion, optimizer, device, epochs=20)

torch.save(scnn_model.state_dict(), "scnn_model.pth")
scnn_model = SCNN()
scnn_model.load_state_dict(torch.load("scnn_model.pth"))
#model.eval()

Epoch 1, Loss: 1.3310, Accuracy: 0.3624
Epoch 2, Loss: 1.2072, Accuracy: 0.4553
Epoch 3, Loss: 1.0808, Accuracy: 0.5274
Epoch 4, Loss: 0.9599, Accuracy: 0.5871
Epoch 5, Loss: 0.8666, Accuracy: 0.6355
Epoch 6, Loss: 0.7862, Accuracy: 0.6747
Epoch 7, Loss: 0.7128, Accuracy: 0.7077
Epoch 8, Loss: 0.6416, Accuracy: 0.7410
Epoch 9, Loss: 0.5522, Accuracy: 0.7783
Epoch 10, Loss: 0.4743, Accuracy: 0.8140
Epoch 11, Loss: 0.3941, Accuracy: 0.8488
Epoch 12, Loss: 0.3173, Accuracy: 0.8800
Epoch 13, Loss: 0.2749, Accuracy: 0.8975
Epoch 14, Loss: 0.2274, Accuracy: 0.9147
Epoch 15, Loss: 0.1876, Accuracy: 0.9302
Epoch 16, Loss: 0.1571, Accuracy: 0.9412
Epoch 17, Loss: 0.1484, Accuracy: 0.9453
Epoch 18, Loss: 0.1328, Accuracy: 0.9527
Epoch 19, Loss: 0.1131, Accuracy: 0.9601
Epoch 20, Loss: 0.1066, Accuracy: 0.9621


/var/folders/tv/91d10mmd5rgbzh1nyczpzq3h0000gn/T/ipykernel_12636/3226149364.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  scnn_model.load_state_dict(torch.load("scnn_

<All keys matched successfully>

In [173]:
# after training both: connect them 

class HybridEmotionRecognizer:
    def __init__(self, pcnn, scnn, device):
        self.pcnn = pcnn.to(device).eval()
        self.scnn = scnn.to(device).eval()
        self.device = device

        # Mapping from S-CNN outputs to real emotion labels
        self.sub_emotion_map = {
            0: 'surprise',  # Happy path
            1: 'neutral',
            2: 'angry',     # Sad path
            3: 'fear'
        }

    def predict(self, image_tensor):
        # image_tensor: (1, 3, 100, 100)
        with torch.no_grad():
            image_tensor = image_tensor.to(self.device)

            # Stage 1: P-CNN
            primary_logits = self.pcnn(image_tensor)
            primary_class = torch.argmax(primary_logits, dim=1).item()  # 0 = happy, 1 = sad

            # Stage 2: S-CNN
            sub_logits = self.scnn(image_tensor)
            sub_class = torch.argmax(sub_logits, dim=1).item()

            # Interpret result based on primary emotion
            print("primary_class:", primary_class)
            if primary_class == 0:  # Happy
                label = self.sub_emotion_map[sub_class if sub_class in [0, 1] else 1] # default neutral
                print("the subclass:", sub_class)
            else:  # Sad
                label = self.sub_emotion_map[sub_class if sub_class in [2, 3] else 2] # default anger
                print("the subclass:", sub_class)

        return {
            "primary": "happy" if primary_class == 0 else "sad",
            "secondary": label
        }


In [124]:
X_final_test[0].shape

(100, 100, 3)

In [125]:
torch.tensor( X_final_test[0])

tensor([[[0.6039, 0.7412, 0.8196],
         [0.6078, 0.7451, 0.8235],
         [0.6196, 0.7569, 0.8353],
         ...,
         [0.0235, 0.0078, 0.0039],
         [0.0235, 0.0078, 0.0039],
         [0.0235, 0.0078, 0.0039]],

        [[0.6039, 0.7412, 0.8196],
         [0.6118, 0.7490, 0.8275],
         [0.6196, 0.7569, 0.8353],
         ...,
         [0.0235, 0.0078, 0.0039],
         [0.0235, 0.0078, 0.0039],
         [0.0235, 0.0078, 0.0039]],

        [[0.6039, 0.7412, 0.8196],
         [0.6118, 0.7490, 0.8275],
         [0.6196, 0.7569, 0.8353],
         ...,
         [0.0235, 0.0078, 0.0039],
         [0.0235, 0.0078, 0.0039],
         [0.0235, 0.0078, 0.0039]],

        ...,

        [[0.7137, 0.8118, 0.8510],
         [0.7137, 0.8118, 0.8510],
         [0.7176, 0.8157, 0.8549],
         ...,
         [0.0941, 0.0980, 0.0902],
         [0.1176, 0.1216, 0.1137],
         [0.1137, 0.1176, 0.1098]],

        [[0.7137, 0.8118, 0.8510],
         [0.7137, 0.8118, 0.8510],
         [0.

In [131]:
torch.tensor(X_final_test[0]).permute(2, 0, 1).unsqueeze(0).float().shape

torch.Size([1, 3, 100, 100])

In [179]:
# Example: run on a single preprocessed image
# Assuming your image is HWC numpy array, 100x100 RGB
for i in X_final_test[:2]:
    image = torch.tensor(i).permute(2, 0, 1).unsqueeze(0).float()  # → (1, 3, 100, 100)

    pipeline = HybridEmotionRecognizer(pcnn_model, scnn_model, device=device)
    result = pipeline.predict(image)
    #print("Primary Emotion:", result["primary"])
    #print("Secondary Emotion:", result["secondary"])


primary_class: 0
the subclass: 1
primary_class: 0
the subclass: 3


In [168]:
set(y_final_test)

{np.str_('0'),
 np.str_('1'),
 np.str_('2'),
 np.str_('3'),
 np.str_('4'),
 np.str_('5'),
 np.str_('6'),
 np.str_('angry'),
 np.str_('disgust'),
 np.str_('fear'),
 np.str_('happy'),
 np.str_('neutral'),
 np.str_('sad'),
 np.str_('surprise')}

In [175]:
raf_to_fer = {
  '1': '5',   # Surprise
  '2': '2',   # Fear
  '3': '1',   # Disgust
  '4': '3',   # Happy
  '5': '4',   # Sad
  '6': '0',   # Angry
  '7': '6'    # Neutral
}

valid_labels = {
    '5': 0,  # Surprise
    '6': 1,  # Neutral
    '0': 2,  # Angry
    '2': 3,   # Fear
    'surprise': 0,  # Surprise
    'neutral': 1,  # Neutral
    'angry': 2,  # Angry
    'fear': 3   # Fear
}


# Map FER label to primary emotion label
def get_primary_label(fer_label):
    if fer_label == 3:
        return 0  # Happy
    elif fer_label == 4:
        return 1  # Sad
    else:
        return None

def evaluate_hybrid(hybrid_model, images, labels):
    correct = 0
    total = 0

    for img, label in zip(images, labels):
        if label not in valid_labels:
            continue  # skip non-SCNN labels

        # Preprocess image
        x = torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float()

        # Predict with hybrid pipeline
        pred = hybrid_model.predict(x)

        # Ground truth
        expected_subclass = valid_labels[label]
        expected_primary = 0 if expected_subclass in [0, 1] else 1  # Surprise/Neutral → Happy, else Sad

        # Convert predicted secondary label to internal index
        pred_subclass = {
            "surprise": 0,
            "neutral": 1,
            "angry": 2,
            "fear": 3
        }[pred["secondary"]]

        pred_primary = 0 if pred["primary"] == "happy" else 1

        if pred_primary == expected_primary and pred_subclass == expected_subclass:
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0
    print(f"Hybrid Test Accuracy (Secondary emotion): {acc:.4f} on {total} samples")


In [180]:
pipeline = HybridEmotionRecognizer(pcnn_model, scnn_model, device)
evaluate_hybrid(pipeline, X_final_test, y_final_test)

primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 1
primary_class: 0
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 0
the subclass: 2
primary_class: 0
the subclass: 1
primary_class: 0
the subclass: 3
primary_class: 1
the subclass: 2
primary_class: 0
the subclass: 1
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 0
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 0
primary_class: 0
the subclass: 2
primary_class: 1
the subclass: 0
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 1
primary_class: 0
the subclass: 1
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 1
the subclass: 2
primary_class: 1
the subclass: 1
primary_class: 0
the subclass: 3
primary_class: 1
the subclass: 3
primary_class: 1
the subclass: 1
primary_class: 0
the subclass: 1
primary_cl